# TP 5 : Tableaux et Complexité

## Petites questions

1. Écrire une fonction `somme` pour calculer la somme des éléments d'un tableau d'entiers.  
2. Écrire une fonction `maximum` pour calculer le maximum d'un tableau d'entiers. On pourra utiliser la fonction `max` renvoyant le maximum de 2 nombres.  
3. Écrire une fonction `list_of_array` transformant un tableau en liste.  
4. Tester si un tableau est trié par ordre croissant.

In [12]:
let somme t =
    let acc = ref 0 in
    for i=0 to (Array.length t) - 1 do
        acc := !acc + t.(i)
    done; !acc;;
    
somme [|1;3;5|];;

val somme : int array -> int = <fun>


- : int = 9


In [13]:
let maximum t =
    let max = ref min_int in
    for i=0 to (Array.length t) -1 do
        if t.(i) > !max then max := t.(i);
    done; !max;;
    
maximum [|1;4;6;3|];;

val maximum : int array -> int = <fun>


- : int = 6


In [14]:
let list_of_array t =
    let l = ref [] in
    for i=((Array.length t) -1) downto 0 do
        l := t.(i)::!l
    done; !l;;
    
list_of_array [|1;5;7;3;5|];;

val list_of_array : 'a array -> 'a list = <fun>


- : int list = [1; 5; 7; 3; 5]


In [15]:
let is_sorted t =
    let sorted = ref true in
    for i=1 to Array.length t - 1 do
        if t.(i) < t.(i-1) then 
            sorted := false; 
    done;
    !sorted;;
    
is_sorted [|1;2;4;4;7|];;

val is_sorted : 'a array -> bool = <fun>


- : bool = true


## Maximum local dans un tableau

Un maximum local dans un tableau `t` est un indice `i` tel que `t.(i) >= t.(i+1)` et `t.(i) >= t.(i-1)`. Pour les extrémités, qu'une seule de ces conditions doit être vérifiée (si `t.(i-1)` ou `t.(i+1)` n'existe pas).  
1. Montrer qu'il existe forcément un maximum local dans `t`.  
2. Écrire une fonction `max_local1` trouvant un maximum local dans un tableau en regardant chaque élément un par un (recherche séquentielle). Quelle est sa complexité ?  
3. Ecrire une fonction `max_local2` faisant la même chose mais avec une méthode par dichotomie (en divisant par 2 la taille du problème à chaque itération), pour avoir une complexité logarithmique.  
*Aide* : soit `t.(m)` le milieu du tableau. Que peut-on dire si `t.(m) < t.(m+1)` ? Si `t.(m) < t.(m-1)` ?

1. Le tableau t contenant obligatoirement un maximum global, ce maximum est également maximum local

In [16]:
let max_local1 t = let m = ref (-1) in
    if t.(0) >= t.(1) then m := 0;
    let n = Array.length t in
    if t.(n-1) > t.(n-2) then t.(n-1) else
    for i=1 to n-1 do
        if t.(i) >= max t.(i-1) t.(i+1)
        then m:=i;
    done; 
    !m;;

val max_local1 : unit array -> int = <fun>


In [25]:
let max_local2 t = 
    let m = Array.length t in
    let rec aux i j =
        let m = (i+j)/2 in
        if (m=0 || t.(m) >= t.(m-1)) && (m=m-1 || t.(m) >= t.(m-1))
        then m
        else if m <> m-1 && t.(m) <= t.(m+1)
        then aux (m+1) j
        else aux i (m-1) in
    aux 0 (m-1)

val max_local2 : 'a array -> int = <fun>


In [17]:
let dicho t e =
    let rec aux i j =
        if i > j then false
        else let m = (i+j)/2 in
            if t.(m) = e then true
            else if t.(m) < e then aux (m+1) j
            else aux i (m-1)
        in aux 0 (Array.length t -1);;
        
dicho [|1;4;5;6;7|] 6

val dicho : 'a array -> 'a -> bool = <fun>


- : bool = true


## Tri par comptage

Écrire une fonction `tri_comptage : ’a array -> unit` qui trie un tableau `t` de taille $n$ dont les entrées sont des entiers entre 0
et $M$ (où $M$ est le maximum de `t`), en complexité O($n + M$).  
Pour cela :  
- Créer un autre tableau `compte` de taille $M + 1$ (avec `Array.make`), initialement rempli de 0
- Remplir `compte` pour que `compte.(i)` contienne le nombre d'apparitions de `i` dans `t`
- Recopier les éléments de `compte` dans `t` dans l'ordre croissant pour obtenir un tableau trié

In [29]:
let tri_comptage t = 
    let m = maximum t in
    let compte = (Array.make (m-1) 0) in
    for i=0 to Array.length t - 1 do
        compte.(t.(i)-1) <- compte.(t.(i)-1) + 1
    done;
    compte;;
    
tri_comptage [|1;4;6;3;6;3;5|];;

val tri_comptage : int array -> int array = <fun>


error: runtime_error

## Tranche maximum

Ecrire une fonction `tranche_max : int array -> int` qui renvoie en O($n$) la somme maximum d'éléments consécutifs d'un tableau de taille $n$. Par exemple, `tranche_max [|1; -2; 6; -3; 2; 4; -8; 7|]` doit renvoyer $9$, correspondant aux éléments 6; -3; 2; 4.  

*Indice* : parcourir le tableau `t` avec une boucle for. Stocker 2 variables `m` et `m_cur` telles que, au $i$ème passage dans la boucle for :  
- `m` contient la somme maximum d'éléments consécutifs parmi les $i$ premiers éléments du tableau
- `m_cur` contient la somme maximum d'éléments consécutifs finissant en `t.(i)`.  
Par exemple, lorsque `tranche_max [|1; -2; 6; -3; 2; 4; -8; 7|]` exécute la $3$ème itération de la boucle `for` (c'est à dire que -3 est considéré), `m` contient 6 (correspondant au seul élément 6) et `m_cur` contient 3 (correspondant à 6; -3).

## Inversions dans un tableau

Étant donné un tableau `t`, une inversion de `t` est un couple d'indices $(i, j)$ tels que $i < j$ et `t`.$(i)$ > `t`.$(j)$. Par exemple, `[|4; 1; 5; 2|]` possède 3 inversions: $(0, 1)$, $(0, 3)$ et $(2, 3)$.
- Écrire une fonction `inv` en O($n^2$) déterminant le nombre d'inversions d'un tableau de taille $n$.
- Écrire une fonction `inv_triee` telle que, si `l1` et `l2` sont deux listes triées par ordre croissant, renvoie le nombre de couples $(i, j)$ tels que `l1.(i) > l2.(j)`.
- (Difficile) Écrire une fonction `inv2` telle que, si `t` est un tableau de taille $n$, `inv2 t` renvoie le nombre d'inversions de `t` en complexité O($n\log(n)$).  
On modifiera le tri fusion pour renvoyer le nombre d'inversions en plus de la liste triée.